In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader, random_split
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'utilizando dispositivo {device}')

utilizando dispositivo cuda


In [4]:
# transformação para o pre processamento das imagens
transform = transforms.Compose([
    transforms.Resize((64,64)),
    transforms.ToTensor(), # converter pra tensor
    transforms.Normalize((0.5,),(0.5,)) #normalização para [-1, 1]
])

In [5]:
dataset = datasets.ImageFolder(root='PetImages', transform=transform)

In [6]:
# definir proporções entre treino e teste
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size

In [8]:
# dividir dados aleatoriamente
train_data, test_data = random_split(dataset, [train_size, test_size])

In [9]:
# criar data loader
batch_size = 32
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle= False)

In [10]:
print(f'tamanho do conjunto de treino: {len(train_data)} imagens')
print(f'tamanho do conjunto de teste: {len(test_data)} imagens')

tamanho do conjunto de treino: 19998 imagens
tamanho do conjunto de teste: 5000 imagens


In [11]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.fc1 = nn.Linear(64 * 16 * 16, 512)
        self.fc2 = nn.Linear(512, 2) # saida de duas classes
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = x.view(-1, 64 * 16 * 16) # flatten
        x = self.relu(self.fc1(x)) 
        x = self.fc2(x) #Removemos Softmax porque CrossEntropyLoss já lida com isso
        return x

In [12]:
# criar a rede e mover pra GPU
modelo = CNN().to(device)

# função de perda e otimizador
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(modelo.parameters(), lr = 0.001)

In [13]:
# treino
def train(modelo, train_loader, criterion, optimizer, epochs = 10):
    modelo.train()
    for epoch in range(epochs):
        running_loss = 0.0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = modelo(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

            print(f"Época {epoch + 1} Loss: {running_loss/len(train_loader):.4f}")


In [21]:
# teste
def test(modelo, test_loader):
    modelo.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = modelo(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            
    print(f"Acurácia: {100 * correct / total:.2f}%")

In [19]:
# treinamento
train(modelo, train_loader, criterion, optimizer, epochs = 10)

Época 1 Loss: 0.0011
Época 1 Loss: 0.0022
Época 1 Loss: 0.0051
Época 1 Loss: 0.0063
Época 1 Loss: 0.0081
Época 1 Loss: 0.0093
Época 1 Loss: 0.0105
Época 1 Loss: 0.0116
Época 1 Loss: 0.0128
Época 1 Loss: 0.0140
Época 1 Loss: 0.0151
Época 1 Loss: 0.0162
Época 1 Loss: 0.0173
Época 1 Loss: 0.0184
Época 1 Loss: 0.0195
Época 1 Loss: 0.0206
Época 1 Loss: 0.0218
Época 1 Loss: 0.0229
Época 1 Loss: 0.0240
Época 1 Loss: 0.0251
Época 1 Loss: 0.0262
Época 1 Loss: 0.0273
Época 1 Loss: 0.0284
Época 1 Loss: 0.0295
Época 1 Loss: 0.0306
Época 1 Loss: 0.0317
Época 1 Loss: 0.0328
Época 1 Loss: 0.0339
Época 1 Loss: 0.0350
Época 1 Loss: 0.0361
Época 1 Loss: 0.0372
Época 1 Loss: 0.0383
Época 1 Loss: 0.0394
Época 1 Loss: 0.0405
Época 1 Loss: 0.0416
Época 1 Loss: 0.0427
Época 1 Loss: 0.0438
Época 1 Loss: 0.0449
Época 1 Loss: 0.0460
Época 1 Loss: 0.0470
Época 1 Loss: 0.0481
Época 1 Loss: 0.0492
Época 1 Loss: 0.0502
Época 1 Loss: 0.0513
Época 1 Loss: 0.0523
Época 1 Loss: 0.0534
Época 1 Loss: 0.0545
Época 1 Loss:

C:\Users\Mateus\anaconda3\Lib\site-packages\PIL\TiffImagePlugin.py:900: UserWarning: Truncated File Read
  warnings.warn(str(msg))


Época 1 Loss: 0.1569
Época 1 Loss: 0.1578
Época 1 Loss: 0.1586
Época 1 Loss: 0.1595
Época 1 Loss: 0.1603
Época 1 Loss: 0.1613
Época 1 Loss: 0.1624
Época 1 Loss: 0.1634
Época 1 Loss: 0.1643
Época 1 Loss: 0.1651
Época 1 Loss: 0.1661
Época 1 Loss: 0.1670
Época 1 Loss: 0.1679
Época 1 Loss: 0.1687
Época 1 Loss: 0.1696
Época 1 Loss: 0.1705
Época 1 Loss: 0.1714
Época 1 Loss: 0.1722
Época 1 Loss: 0.1730
Época 1 Loss: 0.1738
Época 1 Loss: 0.1746
Época 1 Loss: 0.1755
Época 1 Loss: 0.1764
Época 1 Loss: 0.1774
Época 1 Loss: 0.1785
Época 1 Loss: 0.1796
Época 1 Loss: 0.1804
Época 1 Loss: 0.1814
Época 1 Loss: 0.1825
Época 1 Loss: 0.1834
Época 1 Loss: 0.1845
Época 1 Loss: 0.1857
Época 1 Loss: 0.1867
Época 1 Loss: 0.1879
Época 1 Loss: 0.1889
Época 1 Loss: 0.1899
Época 1 Loss: 0.1910
Época 1 Loss: 0.1920
Época 1 Loss: 0.1931
Época 1 Loss: 0.1941
Época 1 Loss: 0.1950
Época 1 Loss: 0.1960
Época 1 Loss: 0.1970
Época 1 Loss: 0.1981
Época 1 Loss: 0.1989
Época 1 Loss: 0.2001
Época 1 Loss: 0.2011
Época 1 Loss:

In [22]:
# teste
test(modelo, test_loader)

Acurácia: 79.20%


In [24]:
torch.save(modelo.state_dict(), "modelo_cachorros_gatos.pth")
print("Modelo salvo com sucesso!")

Modelo salvo com sucesso!
